In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
INPUT_SIZE=10
HIDDEN_SIZE=5
OUTPUT_SIZE=2
BATCH_SIZE=20

In [ ]:
# a two layer rnn
class SimpleRNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleRNN,self).__init__()
        self.rnn=nn.RNN(input_size, hidden_size, batch_first=True)
        self.hidden_size=hidden_size
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax=nn.Softmax(dim=1)
    def forward(self,x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Initialize hidden state
        out, _ = self.rnn(x, h0)  # Get RNN output for all time steps
        out = self.fc(out)  # Apply fully connected layer to each time step
        return out

        
    def init_hidden(self):
        return torch.zeros(1,self.hidden_size)

In [10]:
rnn=SimpleRNN(INPUT_SIZE,HIDDEN_SIZE,OUTPUT_SIZE)
h=rnn.init_hidden()

In [ ]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn
criterion=nn.CrossEntropyLoss()
T=3000
x=torch.rand(BATCH_SIZE,T,INPUT_SIZE)
output=torch.zeros((BATCH_SIZE,T,OUTPUT_SIZE))
print(x)
for i in range(500):
    hidden = rnn.init_hidden()
    rnn.zero_grad()
    x=torch.rand(T,INPUT_SIZE,requires_grad=True)
    optimizer = optim.Adam(rnn.parameters())
    gold_tensor=torch.zeros(T, dtype=torch.long)
    predictions
    optimizer.step()
    print(loss)

# print(loss)
def train(gold_tensor, input_tensor):
    rnn.zero_grad()
    for i in range(T):
        output, hidden = rnn(input_tensor[i], hidden)
    loss = criterion(output, gold_tensor)
    loss.backward(retain_graph=True)
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
    return output, loss.item()


tensor([[[0.8117, 0.6684, 0.0676,  ..., 0.7028, 0.7567, 0.0788],
         [0.4704, 0.7951, 0.9078,  ..., 0.8409, 0.5486, 0.4383],
         [0.3530, 0.6708, 0.3745,  ..., 0.8771, 0.4568, 0.1929],
         ...,
         [0.5084, 0.5360, 0.5377,  ..., 0.3886, 0.7160, 0.4137],
         [0.4758, 0.2053, 0.1933,  ..., 0.1102, 0.6248, 0.0098],
         [0.0692, 0.5833, 0.7944,  ..., 0.3423, 0.0401, 0.1711]],

        [[0.0776, 0.2932, 0.8699,  ..., 0.3744, 0.4937, 0.2988],
         [0.1484, 0.2413, 0.7386,  ..., 0.6112, 0.3057, 0.1031],
         [0.6146, 0.1556, 0.2657,  ..., 0.3628, 0.4610, 0.0488],
         ...,
         [0.4541, 0.0995, 0.8859,  ..., 0.8432, 0.5282, 0.4867],
         [0.6444, 0.0085, 0.3449,  ..., 0.8685, 0.2367, 0.3958],
         [0.7347, 0.4754, 0.2471,  ..., 0.6975, 0.4526, 0.4363]],

        [[0.7008, 0.0871, 0.2066,  ..., 0.6063, 0.3991, 0.4072],
         [0.8020, 0.9648, 0.9956,  ..., 0.4414, 0.0656, 0.1389],
         [0.9090, 0.6608, 0.9912,  ..., 0.9479, 0.4992, 0.

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Step 1: Prepare Variable-Length Sequences
# Example sequences (word embeddings with 5 features per word)
sequences = [
    torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5],
                  [0.5, 0.4, 0.3, 0.2, 0.1],
                  [0.2, 0.3, 0.4, 0.5, 0.6]], dtype=torch.float32),  # Length 3
    torch.tensor([[0.3, 0.4, 0.5, 0.6, 0.7],
                  [0.7, 0.6, 0.5, 0.4, 0.3]], dtype=torch.float32),  # Length 2
    torch.tensor([[0.4, 0.5, 0.6, 0.7, 0.8]], dtype=torch.float32)  # Length 1
]
# Corresponding word-level labels
labels = [
    torch.tensor([1, 0, 1], dtype=torch.long),  # Length 3
    torch.tensor([0, 1], dtype=torch.long),     # Length 2
    torch.tensor([1], dtype=torch.long)         # Length 1
]

# Pad sequences and labels
padded_sequences = pad_sequence(sequences, batch_first=True)  # (batch_size, max_seq_len, input_size)
padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)  # Use -1 as ignore index

# Compute sequence lengths
sequence_lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)

# Step 2: Define the Model
class WordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(WordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for bidirectional
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out

# Hyperparameters
input_size = 5  # Word embedding size
hidden_size = 16
num_classes = 2  # Binary classification
learning_rate = 0.01
num_epochs = 100

# Model, Loss, Optimizer
model = WordRNN(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss(ignore_index=-1)  # Ignore padding in loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 3: Train the Model
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(padded_sequences, sequence_lengths)
    loss = criterion(predictions.view(-1, num_classes), padded_labels.view(-1))  # Flatten for loss calculation
    
    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Step 4: Evaluate the Model
with torch.no_grad():
    predictions = model(padded_sequences, sequence_lengths)
    predictions = predictions.argmax(dim=-1)  # Get predicted class per word
    print("Predictions:")
    print(predictions)


Epoch [10/100], Loss: 0.6205
Epoch [20/100], Loss: 0.4905
Epoch [30/100], Loss: 0.1018
Epoch [40/100], Loss: 0.0109
Epoch [50/100], Loss: 0.0028
Epoch [60/100], Loss: 0.0014
Epoch [70/100], Loss: 0.0009
Epoch [80/100], Loss: 0.0008
Epoch [90/100], Loss: 0.0007
Epoch [100/100], Loss: 0.0006
Predictions:
tensor([[1, 0, 1],
        [0, 1, 0],
        [1, 0, 0]])


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Step 1: Custom Dataset for Large Data
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data 
        self.labels = labels 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

def collate_fn(batch):
    sequences, labels = zip(*batch)
    #I believe we can transform words into embeddings here
    sequences = [torch.tensor(seq, dtype=torch.float32) for seq in sequences]
    labels = [torch.tensor(label, dtype=torch.long) for label in labels]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    padded_labels = pad_sequence(labels, batch_first=True, padding_value=-1)
    lengths = torch.tensor([len(seq) for seq in sequences], dtype=torch.long)
    return padded_sequences, padded_labels, lengths

data = [
    [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]],
    [[0.2, 0.3], [0.4, 0.5]],
    [[0.1, 0.2]]
]
labels = [
    [1, 0, 1],
    [0, 1],
    [1]
]

dataset = LargeDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=64, collate_fn=collate_fn, shuffle=True, num_workers=0)

class LargeWordRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
    
    def forward(self, x, lengths):
        packed_x = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, _ = self.rnn(packed_x)
        out, _ = pad_packed_sequence(packed_out, batch_first=True)
        out = self.fc(out)
        return out

# Step 4: Training Loop
model = LargeWordRNN(input_size=2, hidden_size=64, num_classes=2)
criterion = nn.CrossEntropyLoss(ignore_index=-1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(10): 
    for padded_sequences, padded_labels, lengths in dataloader:
        optimizer.zero_grad()
        outputs = model(padded_sequences, lengths)
        loss = criterion(outputs.view(-1, 2), padded_labels.view(-1))
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 0.7120
Epoch 2, Loss: 0.7087
Epoch 3, Loss: 0.7056
Epoch 4, Loss: 0.7025
Epoch 5, Loss: 0.6996
Epoch 6, Loss: 0.6967
Epoch 7, Loss: 0.6939
Epoch 8, Loss: 0.6912
Epoch 9, Loss: 0.6885
Epoch 10, Loss: 0.6859
